In [1]:
print(1)

1


# 第一问

In [4]:
import numpy as np

def solve_problem_1_multipoint():
    """
    解决数学建模竞赛的问题1（高精度版）。
    使用5个关键目标点来判断遮蔽，计算单枚烟幕弹对导弹M1的有效遮蔽时长。
    """

    # --- 1. 常量与初始条件 (与之前相同) ---

    G = 9.8
    P_M1_INITIAL = np.array([20000.0, 0.0, 2000.0])
    P_FY1_INITIAL = np.array([17800.0, 0.0, 1800.0])
    P_FAKE_TARGET = np.array([0.0, 0.0, 0.0])
    V_M1_SPEED = 300.0
    V_FY1_SPEED = 120.0
    T_DROP = 1.5
    T_EXPLODE_AFTER_DROP = 3.6
    CLOUD_RADIUS = 10.0
    CLOUD_SINK_SPEED = 3.0
    CLOUD_EFFECTIVE_DURATION = 20.0
    
    # --- 新增：定义5个关键目标点 ---
    # 目标半径为7m, 高度为10m, 中心在 y=200, z=5
    P_TRUE_TARGET_POINTS = [
        # 1. 中心点
        np.array([0.0, 200.0, 5.0]),  # Center
        # 2. 轮廓最接近导弹的顶部和底部
        np.array([0.0, 193.0, 10.0]), # Closest-Top (y=200-7)
        np.array([0.0, 193.0, 0.0]),  # Closest-Bottom
        # 3. 轮廓最远离导弹的顶部和底部
        np.array([0.0, 207.0, 10.0]), # Farthest-Top (y=200+7)
        np.array([0.0, 207.0, 0.0])   # Farthest-Bottom
    ]


    # --- 2. 关键事件与向量的预计算 (与之前相同) ---

    V_FY1 = np.array([-V_FY1_SPEED, 0.0, 0.0])
    P_DROP = P_FY1_INITIAL + V_FY1 * T_DROP
    T_DETONATION = T_DROP + T_EXPLODE_AFTER_DROP
    x_det = P_DROP[0] + V_FY1[0] * T_EXPLODE_AFTER_DROP
    y_det = P_DROP[1]
    z_det = P_DROP[2] - 0.5 * G * T_EXPLODE_AFTER_DROP**2
    P_DETONATION = np.array([x_det, y_det, z_det])
    direction_m1 = P_FAKE_TARGET - P_M1_INITIAL
    V_M1 = (V_M1_SPEED / np.linalg.norm(direction_m1)) * direction_m1


    # --- 3. 动态位置与遮蔽计算函数 (核心修改部分) ---

    def get_missile_pos(t):
        return P_M1_INITIAL + V_M1 * t

    def get_cloud_center_pos(t):
        time_since_detonation = t - T_DETONATION
        sink_distance = CLOUD_SINK_SPEED * time_since_detonation
        return P_DETONATION - np.array([0.0, 0.0, sink_distance])

    def is_fully_shielded(t):
        """
        判断在t时刻，是否所有5个目标点的视线都被同时遮蔽。
        """
        missile_pos = get_missile_pos(t)
        cloud_pos = get_cloud_center_pos(t)
        vec_missile_to_cloud = cloud_pos - missile_pos

        # 遍历每一个目标点
        for target_point in P_TRUE_TARGET_POINTS:
            vec_missile_to_target = target_point - missile_pos
            
            # 计算云团中心到该视线的距离
            cross_product_magnitude = np.linalg.norm(np.cross(vec_missile_to_cloud, vec_missile_to_target))
            line_vector_magnitude = np.linalg.norm(vec_missile_to_target)
            
            if line_vector_magnitude == 0: continue
            
            distance = cross_product_magnitude / line_vector_magnitude
            
            # 只要有一个点的视线没有被遮蔽，就立刻返回False
            if distance > CLOUD_RADIUS:
                return False
        
        # 如果循环结束，所有点都被遮蔽，则返回True
        return True


    # --- 4. 通过仿真循环求解遮蔽时长 ---

    time_step = 0.001
    start_time = T_DETONATION
    end_time = T_DETONATION + CLOUD_EFFECTIVE_DURATION
    
    total_shielding_time = 0.0
    shield_start_time = -1.0
    shield_end_time = -1.0
    
    was_shielded_previously = False

    for t in np.arange(start_time, end_time, time_step):
        # 调用新的判断函数
        is_shielded_now = is_fully_shielded(t)
        
        if is_shielded_now:
            total_shielding_time += time_step
        
        if is_shielded_now and not was_shielded_previously:
            shield_start_time = t
        elif not is_shielded_now and was_shielded_previously:
            # 记录第一次遮蔽结束的时间
            if shield_end_time == -1.0:
                 shield_end_time = t

        was_shielded_previously = is_shielded_now

    # --- 5. 输出结果 ---
    
    print("--- 求解结果：问题1 (5点高精度模型) ---")
    print(f"\n模型假设:")
    print(f" 采用5个关键点定义真目标轮廓，要求视线被完全遮蔽。")

    print(f"\n关键事件参数:")
    print(f" 烟幕弹引爆时间: {T_DETONATION:.3f} s")
    print(f" 烟幕弹引爆点坐标: ({P_DETONATION[0]:.3f}, {P_DETONATION[1]:.3f}, {P_DETONATION[2]:.3f})")

    print(f"\n遮蔽效果分析:")
    if total_shielding_time > 0:
        print(f" 有效遮蔽开始于: t ≈ {shield_start_time:.3f} s")
        print(f" 有效遮蔽结束于: t ≈ {shield_end_time:.3f} s")
        print(f"\n 最终结果：有效遮蔽总时长 ≈ {total_shielding_time:.3f} s")
    else:
        print(" 在此更严格的条件下，未能形成有效的完全遮蔽。")

# 运行求解函数
solve_problem_1_multipoint()

--- 求解结果：问题1 (5点高精度模型) ---

模型假设:
 采用5个关键点定义真目标轮廓，要求视线被完全遮蔽。

关键事件参数:
 烟幕弹引爆时间: 5.100 s
 烟幕弹引爆点坐标: (17188.000, 0.000, 1736.496)

遮蔽效果分析:
 有效遮蔽开始于: t ≈ 8.057 s
 有效遮蔽结束于: t ≈ 11.967 s

 最终结果：有效遮蔽总时长 ≈ 3.910 s
